In [1]:
import timeit
import matplotlib.pyplot as plt

In [2]:
def print_frontier(frontier):
    print("\rThe frontier currently has {} nodes.".format(frontier), end='', flush=True)

class VariableObserver:
    def __init__(self, variable):
        self.variable = variable
        self.previous_value = variable
        self.previous_value_length = len(self.previous_value)
        self.max_length = self.previous_value_length
        self.callback = None

    def set_callback(self, callback):
        self.callback = callback

    def check(self):
        current_value = self.variable
        current_frontier_length = len(current_value)
        if current_frontier_length != self.previous_value_length:
            self.previous_value = current_value
            self.previous_value_length = current_frontier_length
            if current_frontier_length > self.max_length:
                self.max_length = current_frontier_length
            if self.callback:
                self.callback(self.max_length)


class Benchmarking:
    def __init__(self, bfs, dfs, a_star, iterative_deeping, beam, hill_climbing, inputs):
        self.bfs = bfs
        self.dfs = dfs
        self.a_star = a_star
        self.iterative_deeping = iterative_deeping
        self.beam = beam
        self.hill_climbing = hill_climbing
        self.inputs = inputs
        self.list_of_functions = [self.bfs, self.dfs, self.a_star, self.iterative_deeping, self.beam, self.hill_climbing]
        self.names = ["BFS", "DFS", "A*", "Iterative Deeping", "Beam", "Hill Climbing"]
        self.exclusion_list = []
        self.execution_time = [-1, -1, -1, -1, -1, -1]
        self.maximum_frontier_size = [-1, -1, -1, -1, -1, -1]
        self.frontier = []
        self.frontier_observer = None
        
    def update_exclusion_list(self, exclusion_list):
        self.exclsion_list = exclusion_list
        
    def function_timer(self, function, inputs, frontier, check, number=1):
        timer = timeit.Timer(lambda: function(*inputs, frontier, check))
        return timer.timeit(number=number)
    
    def frontier_initialization(self, frontier):
        self.frontier = frontier
        self.frontier_observer = VariableObserver(self.frontier)
        self.frontier_observer.set_callback(print_frontier)
        return self.frontier_observer.check
    
    def run_single_function(self, function, name, inputs, i):
        frontier = [];
        check = self.frontier_initialization(frontier)
        time = self.function_timer(function, inputs, frontier, check)
        print(name, " took " + time + " seconds to run")
        self.execution_time[i] = round(time, 2)
        self.maximum_frontier_size[i] = self.frontier_observer.max_length
        
    def reset_execution_time(self):
        self.execution_time = [-1, -1, -1, -1, -1, -1]
        
    def reset_maximum_frontier_size(self):
        self.maximum_frontier_size = [-1, -1, -1, -1, -1, -1]
    
    def benchmark(self):
        self.reset_execution_time()
        self.reset_maximum_frontier_size()
        for i in range(len(self.list_of_functions)):
            if i not in self.exclusion_list:
                self.run_single_function(self.list_of_functions[i], self.names[i], self.inputs[i], i)
                                
    def graph_execution_times_and_frontier_size(self):
        compared_names = [self.names[i] for i in range(len(self.names)) if i not in self.exclusion_list]
        compared_times = [self.execution_time[i] for i in range(len(self.execution_time)) if i not in self.exclusion_list]
        compared_frontier = [self.maximum_frontier_size[i] for i in range(len(self.maximum_frontier_size)) if i not in self.exclusion_list]

        fig, ax1 = plt.subplots()
        ax2 = ax1.twiny()

        bar_widths = [f / max(compared_frontier) for f in compared_frontier]
        blue_bars = ax1.barh(compared_names, compared_times, color='blue', height=0.5, alpha=0.5)
        ax1.set_xlabel('Execution Time (s)')
        red_bars = ax2.barh(compared_names, compared_frontier, color='red', height=0.2, alpha=0.5, linewidth=0)
        ax2.set_xlabel('Maximum Frontier Size')
        ax2.barh(compared_names, bar_widths, left=0, color='none', linewidth=1, edgecolor='k')
        ax1.legend([blue_bars, red_bars], ['Execution Time (s)', 'Maximum Frontier Size'])

        plt.show()
